In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

c:\Users\eria\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from tps_feb_2021.config import config
from tps_feb_2021.utils import save_model, get_run_logdir

In [3]:
data_dir = config['data_dir']

X = pd.read_csv(data_dir + 'processed/X_train.csv')
y = pd.read_csv(data_dir + 'processed/y_train.csv')

In [4]:
X.shape

(300000, 70)

In [5]:
y.shape

(300000, 1)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42)

```
model = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1)
])
```
I tried the above simple model first.  It was able to achieve a low loss pretty easily but was overfitting.  I'll add some regularization in the next iteration.

```
model = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=X_train.shape[1:], kernel_regularizer=keras.regularizers.L2(l=0.003)),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.L2(l=0.003)),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.L2(l=0.003)),
    keras.layers.Dense(1)
])
```
Adding L2 regularization seemed to help with overfitting a bit but now the loss is significantly higher than top [Leaderboard loss levels](https://www.kaggle.com/competitions/tabular-playground-series-feb-2021/leaderboard).  The loss seems stuck at around 0.75 MSE (~0.866 RMSE).  I'll try to continue with a smaller learning rate to see if further improvement might be possible.  learning_rate=0.01 for the above.

Perhaps minor progress with decreasing the learning rate.  I'll now try to use a sigmoid activation function on the output layer and scale it to the target.  I have read that this can often help models converge.

In [8]:
# for sigmoid output rescaling
min_target = y_train.min()
max_target = y_train.max()

In [36]:
model = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=X_train.shape[1:], kernel_regularizer=keras.regularizers.L2(l=0.003)),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.L2(l=0.003)),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.L2(l=0.003)),
    keras.layers.Dense(1, activation = 'sigmoid'),
    keras.layers.Lambda(lambda x : x * (max_target - min_target) + min_target)
])

In [37]:
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(learning_rate=0.01))

In [38]:
#set up for TensorBoard
import os
#root_logdir = os.path.join(os.curdir, 'my_logs')
proj_dir = 'C:/Users/eria/OneDrive - Novozymes A S/Documents/projects/kaggle/tabular_playground_series/tps_feb_2021/'
root_logdir = proj_dir + 'notebooks/my_logs'

def get_run_logdir():
    import time
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'C:/Users/eria/OneDrive - Novozymes A S/Documents/projects/kaggle/tabular_playground_series/tps_feb_2021/notebooks/my_logs\\run_2022_11_04-14_48_01'

In [39]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [40]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/20
7032/7032 [==============================] - 46s 6ms/step - loss: 3.0767 - val_loss: 2.2296
Epoch 2/20
7032/7032 [==============================] - 42s 6ms/step - loss: 1.7478 - val_loss: 1.3880
Epoch 3/20
7032/7032 [==============================] - 46s 7ms/step - loss: 1.1803 - val_loss: 1.0337
Epoch 4/20
7032/7032 [==============================] - 45s 6ms/step - loss: 0.9363 - val_loss: 0.8778
Epoch 5/20
7032/7032 [==============================] - 42s 6ms/step - loss: 0.8311 - val_loss: 0.8077
Epoch 6/20
7032/7032 [==============================] - 46s 7ms/step - loss: 0.7853 - val_loss: 0.7763
Epoch 7/20
7032/7032 [==============================] - 45s 6ms/step - loss: 0.7656 - val_loss: 0.7606
Epoch 8/20
7032/7032 [==============================] - 45s 6ms/step - loss: 0.7570 - val_loss: 0.7558
Epoch 9/20
7032/7032 [==============================] - 45s 6ms/step - loss: 0.7529 - val_loss: 0.7613
Epoch 10/20
7032/7032 [==============================] - 45s 6ms/step - l

Top leaderboard score: rmse = 0.84115 or mse = 0.70753

In [41]:
# models_dir = config['models_dir']
# model.save('./models/01_base_nn_model_L2reg.h5')
# model.save('../models/03_base_nn_model_sigmoid_out.h5')
save_model(model, '03_base_nn_model_sigmoid_out.h5')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               36352     
                                                                 
 dense_13 (Dense)            (None, 512)               262656    
                                                                 
 dense_14 (Dense)            (None, 512)               262656    
                                                                 
 dense_15 (Dense)            (None, 1)                 513       
                                                                 
 lambda (Lambda)             (None, 1)                 0         
                                                                 
Total params: 562,177
Trainable params: 562,177
Non-trainable params: 0
_________________________________________________________________
